<a href="https://colab.research.google.com/github/drixie/Spoon-Knife/blob/master/Polygon_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from datetime import datetime, timedelta
import time 
import pandas
from scipy import stats

apiKey = "PKFAJFLRLZ1IWAOAVFKL"

now = datetime.now().date()
past = (datetime.now() - timedelta(days=100)).date()
aggs = []

while (past < now):
  response = requests.get("https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/hour/{}/{}?apiKey={}".format(past, now, apiKey))
  res = response.json()
  print(res)
  if (res["resultsCount"] > 0): 
    aggs.extend(res["results"])
  time.sleep(0.5)
  past = past + timedelta(days=7)

df = pandas.DataFrame(aggs)

df["t"] = pandas.to_datetime(df["t"], unit="ms")

df.set_index("t", inplace=True)

df.index = df.index.tz_localize("UTC").tz_convert("America/New_York")

df = df.resample("1D", origin="start").agg({
    "o": "first",
    "h": "max",
    "l": "min",
    "c": "last",
    "v": "sum"
})

df = df.dropna()

df

# !pip install polygon-api-client

# from polygon import RESTClient

# def main(): 
#   key = "PKFAJFLRLZ1IWAOAVFKL"
#   with RESTClient(key) as client: 
#     resp = client.stocks_equities_daily_open_close("AAPL", "2021-02-24")
#     print(resp)

# if __name__ == "__main__":
#   main()

{'status': 'ERROR', 'request_id': '8dfe845b9fac52de825664afc830ee7e', 'error': 'Unknown API Key'}


KeyError: ignored

In [ ]:
df = df.reset_index()

#Bollinger bands
df["20sma"] = df["c"].rolling(window=20).mean()
df["stddev"] = df["c"].rolling(window=20).std()
df["lowerband"] = df["20sma"] + (2*df["stddev"])
df["upperband"] = df["20sma"] - (2*df["stddev"])

#Keltner channels
df["tr"] = abs(df["h"] - df["l"])
df["atr"] = df["tr"].rolling(window=20).mean()
df["upperkeltner"] = df["20sma"] + (df["atr"] * 1.5)
df["lowerkeltner"] = df["20sma"] - (df["atr"] * 1.5)

#Donchian line
df["h20"] = df["h"].rolling(window=20).max()
df["l20"] = df["l"].rolling(window=20).min()
df["avghl"] = (df["h20"] + df["l20"])/2
df["donchian"] = df["c"] - (df["20sma"] + df["avghl"])/2

df = df.fillna(1)

def polyf(series): 
  result = np.polyfit(series.index.values, series.values, 1)
  return result[0] * 19

#TTM Squeeze Histogram 
def linreg(series): 
  # slope, intercept, r_value, p_value, std_err = stats.linregress(series.index.values, series.values)
  # return intercept + slope * series.index
  print(df)

df["linreg"] = df["donchian"].rolling(20).apply(linreg, raw=False)

df

                            t        o         h  ...     l20     avghl  donchian
0   2020-11-17 04:00:00-05:00  119.990  120.5300  ...    1.00    1.0000  1.000000
1   2020-11-17 12:00:00-05:00  120.220  120.6741  ...    1.00    1.0000  1.000000
2   2020-11-18 04:00:00-05:00  119.400  119.8800  ...    1.00    1.0000  1.000000
3   2020-11-18 12:00:00-05:00  119.511  119.8200  ...    1.00    1.0000  1.000000
4   2020-11-19 04:00:00-05:00  117.400  118.3800  ...    1.00    1.0000  1.000000
..                        ...      ...       ...  ...     ...       ...       ...
129 2021-02-22 12:00:00-05:00  126.900  127.9600  ...  125.60  131.7385 -6.346023
130 2021-02-23 04:00:00-05:00  125.900  126.0000  ...  118.39  128.1335 -7.881003
131 2021-02-23 12:00:00-05:00  122.120  126.7100  ...  118.39  128.1335 -4.233502
132 2021-02-24 04:00:00-05:00  125.450  127.4900  ...  118.39  127.6900 -4.683935
133 2021-02-24 12:00:00-05:00  124.520  125.7800  ...  118.39  127.6900 -4.037485

[134 rows x 18 

TypeError: ignored